In [ ]:
pip install langchain langchain-openai openai pydantic langsmith ipywidgets

In [ ]:
import json
import ast
from typing import List, Optional
from pydantic import BaseModel
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import ChatPromptTemplate
import ipywidgets as widgets
from IPython.display import display, HTML
from google.colab import files
import warnings
warnings.filterwarnings('ignore')

print("Imports good to go, no errors")

Imports good to go, no errors


In [ ]:
# config
OPENAI_API_KEY = "YOUR_API_KEY"

In [ ]:
# schemas
# Pass 1: Issue Detection
class CodeIssue(BaseModel):
    file: str
    line: int
    category: str
    severity: str
    description: str
    suggestion: str

class Pass1Output(BaseModel):
    issues: List[CodeIssue]

# Pass 2: TODO Tasks
class TODOTask(BaseModel):
    priority: str
    task_description: str
    related_issues: List[int]
    patch_hint: str
    test_recommendation: str

class Pass2Output(BaseModel):
    tasks: List[TODOTask]


In [ ]:
# init llm settings
llm = ChatOpenAI(
    model="gpt-4o-mini",
    openai_api_key=OPENAI_API_KEY,
    temperature=0.2
)

In [ ]:
pass1_parser = PydanticOutputParser(pydantic_object=Pass1Output)

pass1_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a senior software engineer with 10+ years of experience in code review, specializing in Python security, performance optimization, and software architecture.

Your task: Perform a thorough technical code review focusing on:
- BUG_RISK: Logic errors, edge cases, null handling, race conditions
- SECURITY: SQL injection, XSS, authentication flaws, data exposure, input validation
- PERFORMANCE: Inefficient algorithms, memory leaks, unnecessary computations, database N+1 queries
- READABILITY: Naming conventions, code clarity, documentation gaps
- MAINTAINABILITY: Code duplication, tight coupling, violation of SOLID principles

For each issue found:
1. Identify exact file and line number
2. Assign severity (CRITICAL: production-breaking, HIGH: significant impact, MEDIUM: notable issue, LOW: minor improvement)
3. Provide actionable suggestion with specific fix

Output ONLY valid JSON matching the schema. No markdown, no explanations outside JSON."""),

    ("user", """Review the following code changes:

=== CODE BEFORE ===
{code_before}

=== CODE AFTER ===
{code_after}

=== CHANGE DESCRIPTION ===
{description}

Analyze what changed and identify issues introduced or existing problems.

{format_instructions}""")
])

pass1_chain = pass1_prompt | llm | pass1_parser

print("Pass 1 chain ready")

Pass 1 chain ready


In [ ]:
pass2_parser = PydanticOutputParser(pydantic_object=Pass2Output)

pass2_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a technical project lead responsible for converting code review findings into an actionable engineering roadmap.

Your task: Transform identified issues into a prioritized TODO plan that developers can execute immediately.

Prioritization criteria:
- P0 (Critical): Security vulnerabilities, data loss risks, production blockers - fix within 24 hours
- P1 (High): Bugs affecting core functionality, performance degradation - fix within 1 week
- P2 (Medium): Code quality issues, minor bugs, tech debt - fix within 1 sprint
- P3 (Low): Cosmetic improvements, optional refactoring - backlog

For each task:
1. Group related issues together (e.g., all auth issues in one task)
2. Write clear, specific task description with acceptance criteria
3. Provide concrete patch hint showing the fix approach (pseudo-code or example)
4. Recommend test strategy (unit/integration/e2e tests to add)
5. Reference issue indices from the review

Output ONLY valid JSON matching the schema. No markdown, no preamble."""),

    ("user", """Convert these code review issues into actionable tasks:

=== ISSUES FOUND ===
{issues_json}

=== CURRENT CODE ===
{code_after}

Create a prioritized TODO plan. Group related issues. Provide implementation guidance.

{format_instructions}""")
])

llm_pass2 = ChatOpenAI(
    model="gpt-3.5-turbo",
    openai_api_key=OPENAI_API_KEY,
    temperature=0.4  # Slightly higher for creative problem-solving
)

pass2_chain = pass2_prompt | llm_pass2 | pass2_parser

print("Pass 2 chain ready")

Pass 2 chain ready


In [ ]:
def validate_python(code):
    try:
        ast.parse(code)
        return True, ""
    except:
        return False, "Code has syntax errors"

def run_review(code_before, code_after, description):
    # Check if code is valid
    is_valid, error_msg = validate_python(code_after)
    if not is_valid:
        return {"error": error_msg}

    # Run Pass 1
    print("Checking for issues...")
    try:
        pass1_result = pass1_chain.invoke({
            "code_before": code_before,
            "code_after": code_after,
            "description": description,
            "format_instructions": pass1_parser.get_format_instructions()
        })
    except Exception as e:
        return {"error": f"Pass 1 failed: {str(e)}"}

    # Convert issues to JSON string
    issues_list = []
    for issue in pass1_result.issues:
        issues_list.append({
            "file": issue.file,
            "line": issue.line,
            "category": issue.category,
            "severity": issue.severity,
            "description": issue.description,
            "suggestion": issue.suggestion
        })
    issues_json_string = json.dumps(issues_list, indent=2)

    # Run Pass 2
    print("Creating TODO tasks...")
    try:
        pass2_result = pass2_chain.invoke({
            "issues_json": issues_json_string,
            "code_after": code_after,
            "format_instructions": pass2_parser.get_format_instructions()
        })
    except Exception as e:
        return {"error": f"Pass 2 failed: {str(e)}"}

    # Convert tasks to simple format
    tasks_list = []
    for task in pass2_result.tasks:
        tasks_list.append({
            "priority": task.priority,
            "task_description": task.task_description,
            "related_issues": task.related_issues,
            "patch_hint": task.patch_hint,
            "test_recommendation": task.test_recommendation
        })

    # Return everything
    return {
        "issues": issues_list,
        "tasks": tasks_list
    }

print("Review function ready")

Review function ready


In [ ]:
# Input widgets
code_before = widgets.Textarea(
    placeholder='Paste code before changes...',
    layout=widgets.Layout(width='100%', height='200px')
)
code_after = widgets.Textarea(
    placeholder='Paste code after changes...',
    layout=widgets.Layout(width='100%', height='200px')
)
description = widgets.Text(
    placeholder='Brief description of changes',
    layout=widgets.Layout(width='100%')
)
run_button = widgets.Button(
    description='Analyze Code',
    button_style='success'
)
output = widgets.Output()

def on_run_click(b):
    with output:
        output.clear_output()

        if not code_after.value.strip():
            print("❌ Please provide 'code after' at minimum")
            return

        try:
            result = run_review(
                code_before.value or "# No previous code",
                code_after.value,
                description.value or "Code changes"
            )

            if "error" in result:
                print(f"❌ {result['error']}")
                return

            # Save files
            with open('issues.json', 'w') as f:
                json.dump(result['issues'], f, indent=2)
            with open('tasks.json', 'w') as f:
                json.dump(result['tasks'], f, indent=2)

            print(f"✓ Found {len(result['issues'])} issues")
            print(f"✓ Created {len(result['tasks'])} tasks")

            print("\n--- All Issues ---")
            for i, issue in enumerate(result['issues']):
                print(f"{i}. [{issue['severity']}] {issue['description']}")

            print("\n--- All Tasks ---")
            for i, task in enumerate(result['tasks']):
                print(f"{i}. [{task['priority']}] {task['task_description']}")

            print("\n📥 Download files below:")
            files.download('issues.json')
            files.download('tasks.json')

        except Exception as e:
            print(f"❌ Error: {str(e)}")

run_button.on_click(on_run_click)

# Display
display(HTML("<h2>Code Review Assistant</h2>"))
display(widgets.Label("Code Before:"))
display(code_before)
display(widgets.Label("Code After:"))
display(code_after)
display(widgets.Label("Description:"))
display(description)
display(run_button)
display(output)

Label(value='Code Before:')

Textarea(value='', layout=Layout(height='200px', width='100%'), placeholder='Paste code before changes...')

Label(value='Code After:')

Textarea(value='', layout=Layout(height='200px', width='100%'), placeholder='Paste code after changes...')

Label(value='Description:')

Text(value='', layout=Layout(width='100%'), placeholder='Brief description of changes')

Button(button_style='success', description='Analyze Code', style=ButtonStyle())

Output()